# Read Data Sample

In [1]:
import pandas as pd
import numpy as np
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

In [2]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    
    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels.pkl")
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels.pkl")
    

In [3]:
dataset.kdd_train_2labels.shape

(125973, 124)

In [4]:
dataset.kdd_test_2labels.shape

(22544, 124)

In [5]:
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['is_Attack','is_Normal']
    
    x_input = dataset.kdd_train_2labels.drop(output_columns_2labels, axis = 1)
    y_output = dataset.kdd_train_2labels.loc[:,output_columns_2labels]

    x_test_input = dataset.kdd_test_2labels.drop(output_columns_2labels, axis = 1)
    y_test = dataset.kdd_test_2labels.loc[:,output_columns_2labels]

    ss = pp.StandardScaler()

    x_train = ss.fit_transform(x_input)
    x_test = ss.transform(x_test_input)

    y_train = y_output.values
    y_test = y_test.values

    x_train = np.hstack((x_train, y_train))
    x_test = np.hstack((x_test, np.random.normal(size = (x_test.shape[0], y_train.shape[1]))))
    #x_test = np.hstack((x_test, y_test))
    
preprocess.x_train.std()

0.99186991653217393

In [6]:
import tensorflow as tf


In [7]:
class network(object):
    
    input_dim = 124
    classes = 2
    hidden_encoder_dim = 124
    hidden_layers = 1
    latent_dim = 10

    hidden_decoder_dim = 124
    lam = 0.01
    
    def __init__(self, classes, hidden_layers, num_of_features):
        self.classes = classes
        self.hidden_layers = hidden_layers
        self.latent_dim = num_of_features
            
    def build_layers(self):
        tf.reset_default_graph()
        #learning_rate = tf.Variable(initial_value=0.001)

        input_dim = self.input_dim
        classes = self.classes
        hidden_encoder_dim = self.hidden_encoder_dim
        hidden_layers = self.hidden_layers
        latent_dim = self.latent_dim
        hidden_decoder_dim = self.hidden_decoder_dim
        lam = self.lam
        
        with tf.variable_scope("Input"):
            self.x = tf.placeholder("float", shape=[None, input_dim])
            self.y_ = tf.placeholder("float", shape=[None, classes])
            self.keep_prob = tf.placeholder("float")
        
        with tf.variable_scope("Layer_Encoder"):

            hidden_encoder = tf.layers.dense(self.x, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_encoder = tf.layers.dense(hidden_encoder, hidden_encoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_encoder = tf.nn.dropout(hidden_encoder, self.keep_prob)
                
        with tf.variable_scope("Layer_Mean"):
            mu_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = None, kernel_regularizer=tf.nn.l2_loss)

        with tf.variable_scope("Layer_Variance"):
            logvar_encoder = tf.layers.dense(hidden_encoder, latent_dim, activation = None, kernel_regularizer=tf.nn.l2_loss)

        with tf.variable_scope("Sampling_Distribution"):
            # Sample epsilon
            epsilon = tf.random_normal(tf.shape(logvar_encoder), mean=0, stddev=1, name='epsilon')

            # Sample latent variable
            std_encoder = tf.exp(0.5 * logvar_encoder)
            z = mu_encoder + tf.multiply(std_encoder, epsilon)
            
            #tf.summary.histogram("Sample_Distribution", z)

        with tf.variable_scope("Layer_Decoder"):
            hidden_decoder = tf.layers.dense(z, hidden_decoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
            hidden_decoder = tf.nn.dropout(hidden_decoder, self.keep_prob)
            for h in range(hidden_layers - 1):
                hidden_decoder = tf.layers.dense(hidden_decoder, hidden_decoder_dim, activation = tf.nn.relu, kernel_regularizer=tf.nn.l2_loss)
                hidden_decoder = tf.nn.dropout(hidden_decoder, self.keep_prob)
                
        with tf.variable_scope("Layer_Reconstruction"):
            self.x_hat = tf.layers.dense(hidden_decoder, input_dim, activation = None)
            
            self.y = tf.slice(self.x_hat, [0,input_dim-2], [-1,-1])

        with tf.variable_scope("Loss"):
            
            self.regularized_loss = tf.losses.mean_squared_error(self.x, self.x_hat) #tf.reduce_mean((BCE + KLD + softmax_loss) * lam)

            correct_prediction = tf.equal(tf.argmax(self.y, 1), tf.argmax(self.y_, 1))
            self.tf_accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name = "Accuracy")

        with tf.variable_scope("Optimizer"):
            learning_rate=1e-2
            optimizer = tf.train.AdamOptimizer(learning_rate)
            gradients, variables = zip(*optimizer.compute_gradients(self.regularized_loss))
            gradients = [
                None if gradient is None else tf.clip_by_value(gradient, -1, 1)
                for gradient in gradients]
            self.train_op = optimizer.apply_gradients(zip(gradients, variables))
            #self.train_op = optimizer.minimize(self.regularized_loss)
            
        # add op for merging summary
        #self.summary_op = tf.summary.merge_all()
        self.pred = tf.argmax(self.y, axis = 1)
        self.actual = tf.argmax(self.y_, axis = 1)

        # add Saver ops
        self.saver = tf.train.Saver()
        

In [ ]:
import collections

class Train:    
    
    result = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])

    predictions = {}

    results = []
    best_acc = 0
    
    def train(epochs, net, h,f):
        batch_iterations = 200
    
        with tf.Session() as sess:
            #summary_writer_train = tf.summary.FileWriter('./logs/kdd/VAE/training', graph=sess.graph)
            #summary_writer_valid = tf.summary.FileWriter('./logs/kdd/VAE/validation')

            sess.run(tf.global_variables_initializer())

            for epoch in range(1, (epochs+1)):
                print("Step {} | Training Loss:".format(epoch), end = " " )
                x_train, x_valid, y_train, y_valid, = ms.train_test_split(preprocess.x_train, 
                                                                          preprocess.y_train, 
                                                                          test_size=0.1)
                batch_indices = np.array_split(np.arange(x_train.shape[0]), 
                                           batch_iterations)
                                                                          
                for i in batch_indices:
                    _, train_loss = sess.run([net.train_op, 
                                                           net.regularized_loss, 
                                                           ], #net.summary_op
                                                          feed_dict={net.x: x_train[i,:], 
                                                                     net.y_: y_train[i,:], 
                                                                     net.keep_prob:1})
                    
                    #summary_writer_train.add_summary(summary_str, epoch)
                    #if(train_loss > 1e9):
                    
                    print("{:.6f}".format(train_loss), end = ", " )
                    
                print("")
                valid_loss, valid_accuracy = sess.run([net.regularized_loss, net.tf_accuracy], feed_dict={net.x: x_valid, 
                                                                     net.y_: y_valid, 
                                                                     net.keep_prob:1})
                    
                
                accuracy, test_loss, pred_value, actual_value, y_pred = sess.run([net.tf_accuracy, net.regularized_loss, 
                                                               net.pred, 
                                                               net.actual, net.y], 
                                                              feed_dict={net.x: preprocess.x_test, 
                                                                         net.y_: preprocess.y_test, 
                                                                         net.keep_prob:1})
                print("*************** \n")
                print("Step {} | Training Loss: {:.6f} | Test Loss: {:6f} | Test Accuracy: {:.6f}".format(epoch, train_loss, test_loss, accuracy))
                print("*************** \n")
                #print("Accuracy on Test data: {}".format(accuracy))

                curr_pred = pd.DataFrame({"Attack_prob":y_pred[:,-2], "Normal_prob":y_pred[:, -1]})
                Train.predictions.update({"{}_{}_{}".format(epochs,f,h):curr_pred})

                if accuracy > Train.best_acc:
                    Train.best_acc = accuracy
                    Train.pred_value = pred_value
                    Train.actual_value = actual_value
                    Train.best_parameters = "Hidden Layers:{}, Features Count:{}".format(h, f)
                    net.saver.save(sess, "dataset/epochs_{}_hidden layers_{}_features count_{}".format(epochs,h,f))
                Train.results.append(Train.result(epochs, f, h,valid_accuracy, accuracy))


In [ ]:
import itertools
class Hyperparameters:
#    features_arr = [2, 4, 8, 16, 32, 64, 128, 256]
#    hidden_layers_arr = [2, 4, 6, 10]
    features_arr = [4, 8, 16, 32]
    hidden_layers_arr = [2, 4]

    epochs = [10]
    
    for e, h, f in itertools.product(epochs, hidden_layers_arr, features_arr):
        print("Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
        n = network(2,h,f)
        n.build_layers()
        Train.train(e, n, h,f)
        

Current Layer Attributes - epochs:10 hidden layers:2 features count:4
Step 1 | Training Loss: 0.971501, 1.769891, 0.813020, 0.920106, 1.034283, 0.784548, 0.716328, 1.102496, 1.555409, 0.934219, 1.725376, 1.268553, 1.567037, 1.379751, 0.669478, 0.634062, 0.651623, 0.612105, 1.491173, 0.719726, 0.591719, 0.613716, 0.672678, 0.616118, 0.686778, 0.801340, 0.663362, 0.555497, 0.554710, 0.508633, 0.506480, 0.738296, 0.928209, 0.519698, 0.647949, 0.585307, 0.467132, 0.566249, 0.605008, 0.586546, 0.594530, 0.461908, 0.515174, 0.489698, 0.494205, 0.568092, 0.549056, 0.508087, 0.561938, 0.613239, 0.396013, 0.518483, 0.483804, 0.479938, 0.489826, 0.469826, 0.657302, 0.670278, 1.176635, 0.453557, 0.471253, 0.652278, 0.572935, 0.480496, 0.381861, 0.566263, 0.476033, 2.752830, 0.470846, 0.468438, 0.441266, 0.614431, 0.480909, 0.569877, 0.501382, 0.662693, 1.446235, 0.456929, 0.485116, 0.418191, 0.577189, 0.600844, 0.341357, 1.347631, 0.598851, 0.725326, 0.548304, 0.541225, 0.471126, 0.445154, 0.3588

Step 4 | Training Loss: 0.247930, 0.193174, 0.251176, 0.204257, 0.196545, 0.187073, 0.465880, 0.146251, 0.210282, 0.264850, 0.209689, 0.809455, 0.213775, 0.234652, 1.743278, 0.478237, 0.168511, 0.213383, 0.216214, 0.206927, 0.196451, 0.182526, 0.190993, 0.197488, 0.270691, 0.141550, 0.224051, 0.384397, 0.269666, 0.252732, 0.192753, 0.210235, 0.165506, 0.415742, 0.475951, 0.686665, 0.340975, 0.157834, 0.254277, 0.176588, 1.147309, 0.190426, 0.234498, 0.615482, 0.270300, 0.283483, 0.151178, 0.233665, 0.192874, 0.213353, 0.373384, 0.205219, 0.131909, 0.192161, 0.248365, 7.469510, 0.321683, 1.037201, 0.321234, 0.576882, 0.300922, 1.272202, 0.320863, 0.445553, 1.174125, 0.280577, 0.330580, 0.303385, 0.573202, 0.560988, 0.265268, 0.306998, 0.361048, 0.279242, 0.830612, 0.277911, 0.338294, 0.314642, 0.294014, 0.254477, 0.483950, 0.374114, 0.444830, 0.651788, 0.220149, 0.274019, 0.260692, 0.296120, 0.571433, 0.260659, 0.375830, 0.256680, 0.623103, 0.272155, 0.381444, 0.209596, 0.271789, 0.2495

Step 7 | Training Loss: 0.123688, 0.482307, 0.461445, 0.140491, 0.153191, 0.309774, 0.158581, 0.290383, 0.190606, 0.107874, 0.147204, 0.189012, 0.177504, 0.157950, 0.218667, 0.278544, 0.270474, 0.468054, 0.144012, 0.147310, 0.230683, 0.253060, 0.256852, 0.339807, 0.145313, 0.309278, 0.161671, 0.124160, 2.846088, 0.262508, 0.209204, 0.176134, 0.232422, 0.123967, 0.270846, 0.392910, 0.160316, 0.285755, 0.233655, 0.214507, 0.397086, 1.032239, 0.248731, 0.230018, 0.211227, 0.283544, 0.182613, 0.205945, 0.128767, 0.236918, 0.122504, 0.298837, 0.486477, 0.277277, 0.177490, 0.125853, 0.227330, 0.296105, 0.116079, 0.370562, 0.357918, 1.662213, 0.168111, 0.174696, 0.153471, 1.999687, 0.314943, 0.165246, 0.207701, 0.570708, 0.143808, 0.884933, 1.038702, 0.233883, 1.203412, 0.351083, 0.206936, 0.211820, 0.254004, 0.173388, 0.171686, 0.615452, 0.168592, 0.147281, 0.175078, 0.177534, 0.295009, 0.209765, 0.155115, 0.223394, 0.146950, 0.166647, 0.184356, 1.014064, 0.193948, 0.164820, 0.388850, 0.4407

Step 10 | Training Loss: 1.949907, 0.209671, 0.218378, 0.182337, 0.680451, 0.363967, 0.241531, 0.133498, 0.531765, 0.171386, 0.238498, 0.124953, 0.340621, 0.138585, 0.139183, 0.348668, 0.204655, 0.229441, 0.202168, 0.797820, 0.143123, 1.976646, 0.249359, 0.470436, 0.280857, 0.162085, 0.368719, 0.152648, 0.283807, 0.134708, 0.271147, 0.651960, 0.129479, 0.120918, 0.784716, 0.159339, 0.234779, 0.157491, 0.195674, 0.270466, 0.163953, 0.143127, 2.854458, 0.180615, 0.178919, 0.195513, 0.280408, 0.179077, 0.273922, 0.297222, 0.318111, 0.229170, 0.227062, 0.733911, 0.196004, 0.266095, 0.195990, 0.231966, 0.268435, 0.162669, 0.271232, 0.199848, 0.245750, 0.249931, 0.387615, 0.223393, 0.207829, 0.282704, 0.306499, 0.302272, 0.364034, 0.210445, 0.185823, 0.231627, 0.199112, 0.414809, 0.157774, 0.177759, 0.286159, 1.083805, 0.316621, 0.236276, 0.226583, 0.163468, 0.198230, 0.187064, 0.276364, 0.375610, 0.259046, 0.140643, 0.253871, 0.243653, 0.169466, 0.289317, 1.727436, 0.817277, 0.182200, 0.176

Step 3 | Training Loss: 0.513799, 0.713420, 0.336167, 1.390616, 0.383645, 0.571628, 0.358146, 0.628287, 0.317724, 0.330600, 0.307522, 0.361569, 0.365036, 0.271829, 0.297472, 1.790995, 1.395936, 1.284532, 0.407617, 0.411209, 1.217149, 0.500334, 0.353615, 0.493070, 0.381593, 0.365084, 0.370592, 0.365905, 0.566411, 0.332461, 0.395173, 0.404468, 0.348940, 0.545503, 0.304568, 0.555094, 0.364397, 0.377381, 0.695355, 0.414831, 0.442955, 0.355576, 0.351998, 0.474762, 0.332516, 0.550570, 0.700029, 0.392890, 0.339149, 0.438686, 1.021916, 0.320315, 0.288258, 0.443783, 0.592279, 0.292803, 0.383245, 0.360804, 0.879984, 0.309770, 0.238005, 0.342000, 0.367600, 0.342535, 0.554014, 0.311960, 0.655360, 0.331959, 0.307008, 0.669843, 0.599775, 0.354835, 0.475470, 0.313848, 0.288590, 0.283025, 0.323555, 2.452042, 0.279226, 0.365384, 0.336404, 0.397023, 0.397730, 0.474317, 0.440389, 0.510107, 0.317445, 0.288256, 0.339512, 0.525241, 0.281856, 0.315640, 0.393383, 0.281408, 0.401997, 0.336335, 1.010684, 0.4096

...

Step 2 | Training Loss: 0.402886, 0.975248, 0.420045, 0.538008, 2.302425, 0.494261, 0.495515, 1.925852, 0.549273, 0.600623, 0.912084, 0.480389, 0.474792, 0.536496, 0.414477, 0.505838, 0.789487, 0.539340, 0.596916, 0.477188, 2.161722, 0.434707, 0.571569, 0.483956, 0.395496, 0.364864, 0.578889, 0.373908, 0.506640, 0.422862, 0.524012, 0.527440, 0.372570, 0.712442, 0.484792, 0.485658, 0.456206, 0.574511, 0.470867, 0.468654, 1.284800, 0.588425, 0.441773, 0.481813, 0.482475, 0.521844, 0.333427, 0.872651, 0.352700, 0.405941, 0.394562, 0.462857, 0.348386, 0.449446, 0.697969, 0.397796, 0.421934, 0.499786, 0.395840, 0.624960, 0.562013, 0.475002, 0.344053, 0.411357, 0.373651, 0.401817, 0.537667, 0.390775, 0.408198, 0.572425, 0.504801, 0.582764, 0.810840, 0.586848, 0.430362, 0.607379, 0.466047, 0.565665, 0.498292, 0.547518, 0.737951, 1.084204, 0.377886, 0.619785, 0.759707, 0.433715, 0.407608, 0.600968, 0.463154, 3.128324, 0.363220, 0.475882, 0.355480, 0.408724, 0.447550, 0.694620, 0.500376, 0.4111

Step 5 | Training Loss: 0.809966, 0.360138, 0.316869, 0.497697, 0.369341, 0.437427, 0.405874, 0.390062, 0.449411, 0.531350, 0.557400, 0.517632, 0.463326, 0.639807, 0.231688, 0.487348, 0.239436, 0.433378, 0.305651, 1.530137, 0.463119, 0.361785, 0.439754, 0.496278, 0.410367, 0.395010, 0.406378, 0.672883, 0.410565, 0.557038, 0.456693, 0.523996, 0.497751, 0.471716, 0.390639, 0.834977, 0.385161, 0.619583, 0.468638, 0.392329, 0.548967, 0.381730, 0.618245, 0.344423, 0.449898, 0.905424, 0.488749, 0.246218, 0.510070, 0.353474, 0.316702, 0.563176, 0.667877, 0.426993, 0.342167, 0.617098, 0.353339, 0.349368, 0.552150, 0.366163, 0.497497, 0.259049, 1.187636, 0.357329, 0.427976, 0.676901, 0.278771, 0.555886, 0.539154, 0.406785, 0.312391, 0.416162, 5.163030, 0.647494, 0.420133, 0.569597, 0.624100, 0.502813, 0.505877, 0.661151, 1.093920, 0.570800, 0.589593, 1.287169, 0.463359, 0.799281, 0.462621, 0.493599, 0.634355, 0.567432, 0.527261, 0.527557, 0.460385, 0.474897, 0.646703, 0.467656, 0.646545, 0.4643

Step 8 | Training Loss: 0.313387, 0.369292, 0.316891, 0.444941, 0.401543, 0.370301, 0.551082, 0.460531, 0.310568, 0.281237, 0.497335, 0.447667, 0.314692, 0.400460, 0.763992, 0.368275, 0.465546, 0.767869, 3.209454, 0.734301, 0.582828, 0.401817, 0.330567, 0.517643, 0.392048, 0.526324, 0.464855, 0.276697, 0.594282, 0.347221, 0.303080, 0.312798, 1.927690, 0.401206, 0.378478, 0.256407, 0.828013, 0.458359, 0.322291, 0.386560, 0.500117, 0.384966, 0.311025, 0.250126, 0.219678, 0.663573, 0.438609, 0.459145, 0.368795, 0.326882, 0.330125, 0.314108, 0.386702, 0.368969, 0.631455, 0.326939, 0.475361, 0.558862, 1.014912, 0.380577, 1.366403, 0.488567, 0.356489, 0.339970, 0.419172, 0.378786, 0.386787, 0.480082, 0.557727, 0.340228, 0.348515, 0.289092, 0.261778, 0.263606, 0.333967, 0.393476, 0.301479, 0.286323, 0.286501, 0.457545, 0.444532, 0.597281, 0.295234, 0.326284, 0.518962, 0.536155, 0.466872, 1.171396, 0.356079, 0.442129, 0.405210, 0.351185, 0.253262, 0.391567, 0.286481, 0.526335, 0.373302, 0.6392

Step 1 | Training Loss: 0.903183, 0.786761, 1.037285, 0.735378, 0.770248, 4.187818, 1.031164, 0.828462, 0.954487, 0.758738, 1.623254, 1.701246, 1.136253, 0.878385, 0.738636, 0.824601, 0.828447, 0.983785, 0.808081, 0.995275, 0.848099, 0.837081, 0.846488, 0.624719, 0.663153, 0.871296, 0.777110, 0.729604, 2.224874, 0.664879, 0.807169, 0.775805, 0.704947, 0.752185, 0.814644, 0.968777, 0.752416, 0.816308, 0.628423, 0.717592, 1.069137, 0.838317, 0.761214, 0.932092, 1.032774, 1.564557, 0.688969, 0.824416, 0.593936, 0.594289, 0.820447, 0.914490, 1.116049, 0.701959, 1.117843, 0.613412, 0.826143, 0.619142, 0.810047, 0.888098, 0.650844, 1.080411, 0.633569, 0.958261, 1.423971, 0.836392, 0.693081, 0.766321, 0.592552, 0.696914, 0.657019, 0.804416, 0.806448, 0.745992, 0.577049, 0.749529, 0.710553, 0.644751, 0.732646, 0.552740, 0.593514, 0.544113, 0.789736, 0.552428, 0.599306, 1.116099, 0.660911, 0.842922, 0.724566, 0.535011, 0.750488, 0.813540, 0.699827, 0.656423, 0.638419, 0.604455, 0.632687, 0.5667

Step 4 | Training Loss: 0.544561, 0.557070, 2.895801, 0.393129, 2.178460, 0.319510, 0.382530, 0.335877, 0.696253, 0.389558, 0.460700, 0.452024, 0.293900, 3.318252, 0.437133, 0.535576, 0.442195, 0.487128, 0.661142, 0.413512, 0.418947, 0.599508, 0.603113, 0.437176, 0.685286, 0.643349, 0.348111, 0.699664, 1.459856, 0.492756, 0.707126, 0.468905, 0.460375, 0.471011, 0.403054, 0.456855, 0.624108, 0.539216, 0.386195, 0.454893, 0.434717, 0.319705, 0.551122, 0.776016, 0.332568, 0.525415, 0.490020, 0.515477, 0.609498, 0.468847, 0.572941, 0.639245, 0.482408, 0.949967, 0.482101, 0.428406, 0.478470, 0.450920, 0.456973, 0.477098, 0.538017, 0.493732, 0.529845, 0.367082, 0.648129, 0.478543, 0.500964, 0.650224, 0.543782, 1.174892, 0.861282, 0.415794, 0.415056, 0.384234, 0.730778, 0.505852, 0.425767, 0.407080, 0.462706, 0.505953, 0.598769, 0.781586, 0.778582, 0.528543, 1.205656, 0.417199, 0.523034, 0.401974, 0.487633, 0.451136, 0.427954, 0.474092, 0.384783, 0.565120, 0.476767, 0.570057, 0.530617, 0.3868

In [ ]:
df_results = pd.DataFrame(Train.results)

In [ ]:
df_results.sort_values(by = 'test_score', ascending = False)

In [ ]:
pd.Panel(Train.predictions).to_pickle("dataset/tf_vae_only_nsl_kdd_predictions.pkl")
df_results.to_pickle("dataset/tf_vae_only_nsl_kdd_scores.pkl")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    np.set_printoptions(precision=4)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j].round(4),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot(actual_value, pred_value):
    from sklearn.metrics import confusion_matrix
    cm_2labels = confusion_matrix(y_pred = pred_value, y_true = actual_value)
    plt.figure(figsize=[6,6])
    plot_confusion_matrix(cm_2labels, preprocess.output_columns_2labels, normalize = True,
                         title = Train.best_parameters)

In [ ]:
plot(actual_value = Train.actual_value, pred_value = Train.pred_value)